### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, mean,expr, avg, stddev
from pyspark.sql.functions import lag, coalesce, lit
from pyspark.sql.functions import corr
from pyspark.sql.functions import to_date, date_format
from pyspark.sql.window import Window

### Data Investiagtion and Preprocessing

In [2]:
import findspark

# Initialize findspark
findspark.init()

In [3]:
# Create a SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Netflix Stock Price Forecasting") \
    .getOrCreate()
    
# Load the data
df = spark.read.csv("C:/Users/ADMIN/Desktop/Scalable and Distributed Computing/Stock-Price-Forecasting-System-Using-Apache-Spark/data/raw/NFLX.csv", header=True, inferSchema=True)
df.show(5)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [21]:
df.printSchema()
print(f"Rows: {df.count()}, Columns: {len(df.columns)}")

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

Rows: 1009, Columns: 7


In [22]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|              Open|              High|               Low|             Close|         Adj Close|           Volume|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|              1009|              1009|              1009|              1009|              1009|             1009|
|   mean|419.05967286223984|425.32070308027744|412.37404380178384| 419.0007329207132| 419.0007329207132| 7570685.03468781|
| stddev|108.53753170401458|109.26295957119454|107.55586739006031|108.28999877034995|108.28999877034995|5465535.225689975|
|    min|        233.919998|        250.649994|        231.229996|        233.880005|        233.880005|          1144000|
|    max|        692.349976|         700.98999|        686.090027|        691.690002|        691.690002|         58904300|
+-------+-------

In [23]:
# Check for missing values and remove them
missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
missing_values.show()
df = df.dropna()

+----+----+----+---+-----+---------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|
+----+----+----+---+-----+---------+------+
|   0|   0|   0|  0|    0|        0|     0|
+----+----+----+---+-----+---------+------+



In [24]:
# Convert the Date column to Timstamp
df = df.withColumn("Date", col("Date").cast("timestamp"))
# Only show "yyyy-MM-dd" in the Date column
df = df.withColumn("Date", date_format(col("Date"),"yyyy-MM-dd"))

df = df.orderBy("Date")

df.show(5)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2018-02-05|     262.0|267.899994|250.029999|254.259995|254.259995|11896100|
|2018-02-06|247.699997|266.700012|     245.0|265.720001|265.720001|12595800|
|2018-02-07|266.579987|272.450012|264.329987|264.559998|264.559998| 8981500|
|2018-02-08|267.079987|267.619995|     250.0|250.100006|250.100006| 9306700|
|2018-02-09|253.850006|255.800003|236.110001|249.470001|249.470001|16906900|
+----------+----------+----------+----------+----------+----------+--------+
only showing top 5 rows



In [25]:
# Calculate split index
split_index = int(df.count() * 0.8)

# Split the dataset into training and testing sets
train = df.limit(split_index)  # Take the first 80% of rows
test = df.subtract(train)      # Subtract the training set from the original DataFrame to get the test set
test_copy = test.select("*") 
# Display row counts of the resulting DataFrames to verify the split
print(f"Training set row count: {train.count()}")
print(f"Testing set row count: {test.count()}")

Training set row count: 807
Testing set row count: 202


In [26]:
import os

os.makedirs("../data/processed", exist_ok=True)

In [27]:
test_pandas = test.toPandas()
train_pandas = train.toPandas()


# Convert the Date column to datetime format for accurate sorting
test_pandas["Date"] = pd.to_datetime(test_pandas["Date"])
train_pandas["Date"] = pd.to_datetime(train_pandas["Date"])

# Sort the Pandas DataFrame by the Date column
test_pandas = test_pandas.sort_values(by="Date")
train_pandas = train_pandas.sort_values(by="Date")

test_pandas.to_csv("../data/processed/test.csv", index=False)
train_pandas.to_csv("../data/processed/train.csv", index=False)
